In [85]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from google.colab import drive

In [86]:
# 1. 데이터 불러오기
drive.mount('/content/drive')
base_path = '/content/drive/My Drive/Colab Notebooks/homerun/'
gstation_path = f'{base_path}gStation2.csv'
arrival_data_path = f'{base_path}arrival_data2.csv'


gstation_df = pd.read_csv(gstation_path)
arrival_df = pd.read_csv(arrival_data_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
# 2. arrival_data 전처리
# 시간을 분으로 변환하는 함수
def time_to_minutes(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

# 도착 시간을 분으로 변환 (타겟 변수)
if arrival_df['ARRIVAL_TIME'].dtype == 'object':
    arrival_df['ARRIVAL_MINUTES'] = arrival_df['ARRIVAL_TIME'].apply(time_to_minutes)

# 요일과 출발지를 원-핫 인코딩 (모든 요일 포함)
# 가능한 모든 요일과 출발지 값 지정
days = ['MON', 'TUE', 'WED', 'THU', 'FRI']
depart_locations = arrival_df['DEPART_AT'].unique()

# 원-핫 인코딩 수행
arrival_df_encoded = pd.get_dummies(arrival_df, columns=['DAY', 'DEPART_AT'], prefix=['DAY', 'DEPART_AT'])

# 누락된 요일 컬럼 추가
for day in days:
    col_name = f'DAY_{day}'
    if col_name not in arrival_df_encoded.columns:
        arrival_df_encoded[col_name] = False

In [88]:
arrival_df_encoded.head()

,ARRIVAL_TIME,ARRIVAL_MINUTES,DAY_MON,DEPART_AT_SCH,DEPART_AT_STA,DAY_TUE,DAY_WED,DAY_THU,DAY_FRI
0,8:15,495,True,False,True,False,False,False,False
1,8:17,497,True,False,True,False,False,False,False
2,10:00,600,True,True,False,False,False,False,False
3,10:05,605,True,True,False,False,False,False,False
4,10:16,616,True,False,True,False,False,False,False


In [89]:
# 3. 특성과 타겟 분리
feature_columns = ['ARRIVAL_MINUTES', 'DAY_MON', 'DAY_TUE', 'DAY_WED', 'DAY_THU', 'DAY_FRI',
                  'DEPART_AT_SCH', 'DEPART_AT_STA']
X = arrival_df_encoded[feature_columns]  # 특성 순서 고정
y = arrival_df_encoded['ARRIVAL_MINUTES']

In [90]:
# 4. 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [91]:
# 5. 랜덤 포레스트 모델 학습
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42
)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, random_state=42)

In [92]:
# 6. 모델 평가
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"Average prediction error: {np.sqrt(mse):.2f} minutes")

Mean Squared Error: 577.40
Mean Absolute Error: 16.61
Average prediction error: 24.03 minutes


In [94]:
# 7. 새로운 스케줄 생성
days = ['MON', 'TUE', 'WED', 'THU', 'FRI']
new_schedule = gstation_df.copy()

for day in days:
    # 학교 출발 시간에 대한 역 도착 시간 예측
    school_times = gstation_df[f'{day}_SCHOOL_DEPART'].copy()

    for idx, time in school_times.items():
        # 출발 시간을 분으로 변환
        depart_minutes = time_to_minutes(time)

        # 예측을 위한 데이터프레임 생성 (feature_columns와 동일한 순서로)
        pred_data = pd.DataFrame({
            'ARRIVAL_MINUTES': [depart_minutes],
            'DAY_MON': [1 if day == 'MON' else 0],
            'DAY_TUE': [1 if day == 'TUE' else 0],
            'DAY_WED': [1 if day == 'WED' else 0],
            'DAY_THU': [1 if day == 'THU' else 0],
            'DAY_FRI': [1 if day == 'FRI' else 0],
            'DEPART_AT_SCH': [1],
            'DEPART_AT_STA': [0]
        })[feature_columns]  # feature_columns 순서로 정렬

        # 예측
        predicted_minutes = int(rf_model.predict(pred_data)[0])
        predicted_hours = predicted_minutes // 60
        predicted_mins = predicted_minutes % 60

        # 예측된 시간을 스케줄에 업데이트
        new_schedule.at[idx, f'{day}_STATION_DEPART'] = f"{predicted_hours:02d}:{predicted_mins:02d}"


In [95]:
# 8. 새로운 스케줄 저장
output_path = f'{base_path}new_schedule.csv'
new_schedule.to_csv(output_path, index=False)
print("새로운 스케줄이 생성되어 저장되었습니다!")

# 9. 새로운 스케줄 확인
print("\n새로운 스케줄의 처음 5행:")
print(new_schedule.head())

새로운 스케줄이 생성되어 저장되었습니다!

새로운 스케줄의 처음 5행:
  MON_SCHOOL_DEPART MON_STATION_DEPART TUE_SCHOOL_DEPART TUE_STATION_DEPART  \
0              7:45              09:05              7:45              09:05   
1              8:00              09:05              8:00              09:05   
2              8:02              09:05              8:02              09:05   
3              8:03              09:05              8:03              09:05   
4              8:05              09:05              8:05              09:05   

  WED_SCHOOL_DEPART WED_STATION_DEPART THU_SCHOOL_DEPART THU_STATION_DEPART  \
0              7:45              09:05              7:45              09:05   
1              8:00              09:05              8:00              09:05   
2              8:02              09:05              8:02              09:05   
3              8:03              09:05              8:03              09:05   
4              8:05              09:05              8:05              09:05   

  FRI_SCHO